In [2]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [19]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [20]:
mnist_train = dsets.MNIST(root='MNIST_data/',
                        train = True,
                        transform = transforms.ToTensor(),
                        download = True)
mnist_test = dsets.MNIST(root='MNIST_data/',
                        train = False,
                        transform = transforms.ToTensor(),
                        download = True)

In [21]:
data_loader = torch.utils.data.DataLoader(dataset = mnist_train,
                                         batch_size = batch_size,
                                         shuffle=True,
                                         drop_last=True)

In [22]:
class CNN(torch.nn.Module):
    
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1,32,kernel_size = 3, stride = 1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size = 2, stride=2))
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32,64,kernel_size = 3, stride = 1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size = 2, stride=2))
        self.fc = torch.nn.Linear(7*7*64, 10, bias = True)
        torch.nn.init.xavier_uniform(self.fc.weight)
    
    def forward(self,x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out      
        
        

In [23]:
model = CNN().to(device)

<ipython-input-22-36ac61e48484>:14: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  torch.nn.init.xavier_uniform(self.fc.weight)


In [24]:
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [25]:
total_batch = len(data_loader)
print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    avg_cost = 0
    
    for X,Y in data_loader:
        X = X.to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
    print('[Epoch : {:>4}]cost = {:>.9}'.format(epoch+1, avg_cost))

Learning started. It takes sometime.


C:\Users\gicht\anaconda3\envs\deep_class\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


[Epoch :    1]cost = 0.224138841
[Epoch :    2]cost = 0.0595019907
[Epoch :    3]cost = 0.0438317247
[Epoch :    4]cost = 0.0341786034
[Epoch :    5]cost = 0.0274803471
[Epoch :    6]cost = 0.0235629585
[Epoch :    7]cost = 0.0199476983
[Epoch :    8]cost = 0.0165845826
[Epoch :    9]cost = 0.0135253714
[Epoch :   10]cost = 0.0116857784
[Epoch :   11]cost = 0.00954614393
[Epoch :   12]cost = 0.00929218065
[Epoch :   13]cost = 0.00775190303
[Epoch :   14]cost = 0.00609801896
[Epoch :   15]cost = 0.00624407874


In [26]:
with torch.no_grad():
    X_test = mnist_test.test_data.reshape(len(mnist_test), 1, 28,28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

C:\Users\gicht\anaconda3\envs\deep_class\lib\site-packages\torchvision\datasets\mnist.py:67: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
C:\Users\gicht\anaconda3\envs\deep_class\lib\site-packages\torchvision\datasets\mnist.py:57: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy: 0.988099992275238
